In [81]:
import pandas as pd

## Read all the data

In [82]:
left=pd.read_csv('karolinska.csv')
pmcoaki=pd.read_csv('./pmcoalist_ki.csv')
pmcoaki = pmcoaki.rename(columns={'PMID':'pmid'})

left=left.dropna(subset=['is_open_code','is_open_data','is_register_pred','is_coi_pred','is_fund_pred'])
left=pd.merge(left,pmcoaki,on='pmid')
left=left[left.is_research_x]
columns=['pmid','is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Publication Year','Institution','Journal/Book']
left=pd.DataFrame(left, columns=columns)

In [83]:
left

,pmid,is_coi_pred,is_register_pred,is_fund_pred,is_open_code,is_open_data,Publication Year,Institution,Journal/Book
0,28119597,True,False,True,False,True,2017,Karolinska,Front Aging Neurosci
1,28119597,True,False,True,False,True,2017,Karolinska,Front Aging Neurosci
2,28119597,True,False,True,False,True,2017,Karolinska,Front Aging Neurosci
3,28119597,True,False,True,False,True,2017,Karolinska,Front Aging Neurosci
4,28119597,True,False,True,False,True,2017,Karolinska,Front Aging Neurosci
...,...,...,...,...,...,...,...,...,...
16436,35280800,True,False,True,False,False,2022,Karolinska,Front Oncol
16437,35276442,False,False,True,False,True,2022,Karolinska,Redox Biol
16438,35160653,True,False,True,True,True,2022,Karolinska,Materials (Basel)
16439,33743481,True,False,True,False,False,2021,Karolinska,Eur J Cancer


In [84]:
right=pd.read_csv('kidept_pmids_orgbased_2010_2022.csv')

In [85]:
df=left.set_index('pmid').join(right.set_index('pmid'), how='inner')
df['is_open_code']=df['is_open_code'].astype('bool')
df['is_open_data']=df['is_open_data'].astype('bool')

In [86]:
df=df[df['Publication Year'].isin([2017,2018,2019,2020,2021])]

In [87]:
df

,is_coi_pred,is_register_pred,is_fund_pred,is_open_code,is_open_data,Publication Year,Institution,Journal/Book,name,short_name
pmid,,,,,,,,,,
28086826,True,False,True,False,False,2017,Karolinska,BMC Nephrol,"Department of Clinical Science, Intervention a...",CLINTEC
28086826,True,False,True,False,False,2017,Karolinska,BMC Nephrol,"Department of Clinical Science, Intervention a...",CLINTEC
28087486,False,False,True,False,True,2017,Karolinska,Acta Biomater,Department of Medical Biochemistry and Biophysics,MBB
28087510,True,False,True,False,False,2017,Karolinska,BMJ,Department of Medical Epidemiology and Biostat...,MEB
28087510,True,False,True,False,False,2017,Karolinska,BMJ,Department of Medical Epidemiology and Biostat...,MEB
...,...,...,...,...,...,...,...,...,...,...
35052388,True,False,True,False,False,2021,Karolinska,Genes (Basel),Department of Medical Epidemiology and Biostat...,MEB
35053180,True,False,True,False,False,2021,Karolinska,Biomolecules,Department of Oncology-Pathology,OnkPat
35149535,True,True,True,False,False,2021,Karolinska,BMJ Open,Department of Laboratory Medicine,LABMED


## Remove institutions with too few publications

In [88]:
remove_low=pd.DataFrame(df.groupby([ "Publication Year", "short_name"] ).size().reset_index())
indices=remove_low[0][lambda x: x>50].index.tolist()
remove_low=remove_low.iloc[indices]
inst=set(remove_low.short_name)
inst=list(inst)

df_clean=df[df['short_name'].isin(inst)]

In [91]:
round(df_clean.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('general.csv')

# Map to specific groupings

In [94]:
ki_nord=df_clean[df_clean.short_name.isin(['GPH','CNS', 'KI DS','KBH','LIME','LIME', 'MedS','MMK','OnkPat'])]
ki_syd=df_clean[df_clean.short_name.isin(['BioNut','KI SÖS','CLINTEC','LabMed','MedH'])]
ki_solna=df_clean[df_clean.short_name.isin(['CMB','FyFa','MBB','MEB','MTC','IMM','Neuro','NVS','DentMed'])]

In [96]:
round(ki_nord.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_nord.csv')
round(ki_syd.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_syd.csv')
round(ki_solna.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_solna.csv')
